### Generate the mimic metadata, sampled_clinical_data.json

##### REMEMBER TO CHANGE THE DIRECTORY WHEN READING THE DATA AND STORING THE DATA !!!

Update the sample_metadata.json file from the last intake

### Generate the metadata

In [ ]:
# Start updating the sample_metadata_ori.json made by the previous intake.
import json
with open(r"/home/kaop/wehi/demo_env1/sample_metadata_ori.json") as d:
    ori_metadata = json.load(d)


for i in range(len(ori_metadata)):
    if 'ICGC' in ori_metadata[i]['Sample ID']:
        ori_metadata[i]['Sample ID'] = 'LC_Sample' + str(i + 1)

with open(r"/home/kaop/wehi/demo_env1/sample_metadata.json", "w") as d:
    json.dump(ori_metadata, d, indent = 3)


In [ ]:
# Expand the sample size from 100 to 500, and no.of patients remain the same.
import json
import random
import copy
with open(r"/home/kaop/wehi/demo_env1/sample_metadata.json") as d:
    ori_metadata = json.load(d)

# create list for both sample_ids and patienet_ids
sample_ids = []; patient_ids = []
for i in range(len(ori_metadata)):
    sample_ids.append(ori_metadata[i]['Sample ID'])
    patient_ids.append(ori_metadata[i]['Patient ID'])

# generate new samples for each patient
add_metadata = []
for i in range(101, 501, 1):  # Generate the new samples from LC_Sample101 to LC_Sample500.
    patient_belong = random.sample(patient_ids, 1)[0]
    sample_name = 'LC_Sample' + str(i)
    idx = ''
    for i in range(len(ori_metadata)):
        if ori_metadata[i]['Patient ID'] == patient_belong:
            idx = i
            
    adding_metadata = copy.deepcopy(ori_metadata[idx])
    adding_metadata['Sample ID'] = sample_name
    adding_metadata['Number of Samples Per Patient'] += 1
    add_metadata.append(adding_metadata)

# update the original metadata
curr_metadata = ori_metadata + add_metadata


with open(r"/home/kaop/wehi/demo_env1/sample_metadata.json", "w") as f:
    json.dump(curr_metadata, f, indent = 3)
    

### Start generating the mimic counts file

In [ ]:
with open(r"/home/kaop/wehi/demo_env1/sample_metadata.json") as f:
    curr_metadata = json.load(f)

# generate new list for sampleID
sample_ids = []
for i in range(len(curr_metadata)):
    sample_ids.append(curr_metadata[i]['Sample ID'])

In [ ]:
# create a table for sampleID and Patient ID
import pandas as pd
import numpy as np
cortb = pd.DataFrame(columns=['Sample ID', 'Patient ID'])


for patient_id in patient_ids:
    df_temp = pd.DataFrame(columns=['Sample ID', 'Patient ID'])
    
    sample_collects = []
    for i in range(len(curr_metadata)):
            if curr_metadata[i]['Patient ID'] == patient_id:
                sample_collects.append(curr_metadata[i]['Sample ID'])
    df_temp['Sample ID'] = sample_collects
    df_temp['Patient ID'] = patient_id
    cortb = pd.concat([cortb, df_temp], ignore_index=True)

                  

In [35]:
# store the cortb, which is the table with sample and their corresponding patient.
import random
n_genes = 100
samples_lst = list(cortb['Sample ID'])


# generate two datasets
for i in range(2):
    sampled = random.sample(samples_lst, n_genes)
    sample_lst = [sample for sample in samples_lst if sample not in sampled]

    # build mimic counts dataset
    df = pd.DataFrame(np.round(np.random.uniform(0.0, 10.0, size = (len(sampled), n_genes)), 2))
    df.columns = [ 'gene_' + str(i) for i in range(n_genes)]
    df.index = sampled

    # IMPORTANT!! PLEASE CHANGE THE DIRECTORY HERE TO STORE YOUR FILES
    df.to_csv("/home/kaop/wehi/demo_env1/raw_files/counts_dataset" + str(i + 1) + ".csv")

# creating mapping table
with open(r"/home/kaop/wehi/demo_env1/sample_to_patient.json", 'w') as f:
    json.dump(cortb.set_index('Sample ID')['Patient ID'].to_dict(), f, indent=3)
